In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# enables inline plots
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 3)

In [2]:
# import file
import zipfile
local_path = '/Users/eloiseheydenrych/Downloads'
z = zipfile.ZipFile(local_path + '/test_users.csv.zip')
df = pd.read_csv(z.open('test_users.csv'),parse_dates=[1,2])
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 62096 entries, 0 to 62095
Data columns (total 15 columns):
id                         62096 non-null object
date_account_created       62096 non-null datetime64[ns]
timestamp_first_active     62096 non-null datetime64[ns]
date_first_booking         0 non-null float64
gender                     62096 non-null object
age                        33220 non-null float64
signup_method              62096 non-null object
signup_flow                62096 non-null int64
language                   62096 non-null object
affiliate_channel          62096 non-null object
affiliate_provider         62096 non-null object
first_affiliate_tracked    62076 non-null object
signup_app                 62096 non-null object
first_device_type          62096 non-null object
first_browser              62096 non-null object
dtypes: datetime64[ns](2), float64(2), int64(1), object(10)
memory usage: 7.6+ MB


In [3]:
df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,2014-07-01 00:00:06,NaN,FEMALE,35,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,2014-07-01 00:00:51,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,2014-07-01 00:01:48,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,2014-07-01 00:02:15,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,2014-07-01 00:03:05,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [4]:
# create new column with all ones
df['entered_age'] = 1 

# fill in zeros in entered_age where the age column was NaN
df.loc[np.isnan(df['age']), 'entered_age'] = 0

df.head(20)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,entered_age
0,5uwns89zht,2014-07-01,2014-07-01 00:00:06,NaN,FEMALE,35,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,1
1,jtl0dijy2j,2014-07-01,2014-07-01 00:00:51,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari,0
2,xx0ulgorjt,2014-07-01,2014-07-01 00:01:48,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome,0
3,6c6puo6ix0,2014-07-01,2014-07-01 00:02:15,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE,0
4,czqhjk3yfe,2014-07-01,2014-07-01 00:03:05,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari,0
5,szx28ujmhf,2014-07-01,2014-07-01 00:03:36,NaN,FEMALE,28,basic,0,en,sem-brand,google,omg,Web,Windows Desktop,Chrome,1
6,guenkfjcbq,2014-07-01,2014-07-01 00:05:14,NaN,MALE,48,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,1
7,tkpq0mlugk,2014-07-01,2014-07-01 00:06:49,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,0
8,3xtgd5p9dn,2014-07-01,2014-07-01 00:08:37,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,0
9,md9aj22l5a,2014-07-01,2014-07-01 00:22:45,NaN,-unknown-,NaN,basic,0,en,sem-non-brand,google,omg,Web,Windows Desktop,Firefox,0


In [5]:
#create month columns
df['Month_Account_Created'] = df['date_account_created'].dt.month
df['Month_First_Active'] = df['timestamp_first_active'].dt.month
# note -- other things could be tried here. Maybe people who book at 4am choose different
#destinations than those who book at 1pm?, etc.
del df['date_first_booking']